In [1]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import json

In [2]:
es = Elasticsearch("http://10.167.65.50:9200") 

In [ ]:
#Q121337973

In [20]:
query = {
    "query": {
        "match": {
            "meta.id": "Q121337973"
        }
    }
}

response = es.search(index="eye-raw-data", body=query)

print(response)

{'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 4.0253515, 'hits': [{'_index': 'eye-raw-data', '_type': '_doc', '_id': '4e9nlo8BkSW7a5nmB-__', '_score': 4.0253515, '_ignored': ['event.original.keyword'], '_source': {'tags': ['eye-raw-data'], 'event': {'original': '{"meta":{"timestamp":1716215285,"source":"https://www.wikidata.org/wiki/Q121337973","name":"laphonza-butler","id":"Q121337973"},"data":[{"name":"instance of[0]","data":"human"},{"name":"image[0]","data":"Laphonza Butler Senate photo, 2023.jpg\\n5,805 × 7,256; 24.08 MB"},{"name":"image[0].source","data":"https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Laphonza_Butler_Senate_photo%2C_2023.jpg/220px-Laphonza_Butler_Senate_photo%2C_2023.jpg"},{"name":"image[0].point in time","data":"2023"},{"name":"signature[0]","data":"Laphonza Butler signature.svg\\n1,008 × 229; 15 KB"},{"name":"signature[0].source","dat

In [173]:
# Q121337973, Q58106, Q554792 (only wiki)
response

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 4.536177, 'hits': [{'_index': 'eye-raw-data', '_type': '_doc', '_id': '6d-6h48BkSW7a5nmoNJF', '_score': 4.536177, '_ignored': ['data.data.keyword', 'event.original.keyword'], '_source': {'tags': ['eye-raw-data'], '@timestamp': '2024-05-17T18:05:03.648181468Z', '@version': '1', 'meta': {'id': 'Q554792', 'timestamp': 1715958281, 'source': 'https://en.wikipedia.org/wiki/Michael_Bennet', 'name': 'michael-bennet'}, 'event': {'original': '{"meta":{"timestamp":1715958281,"source":"https://en.wikipedia.org/wiki/Michael_Bennet","name":"michael-bennet","id":"Q554792"},"data":[{"name":"","data":""},{"name":"Preceded by","data":"Ken Salazar"},{"name":"Leader","data":"Harry Reid"},{"name":"Preceded by","data":"Patty Murray"},{"name":"Succeeded by","data":"Jon Tester"},{"name":"Preceded by","data":"Jerome Wartgow"},{"nam

In [86]:
import re

def filter_wikipedia_results(response):
    wikipedia_results = []
    for hit in response["hits"]["hits"]:
        source = hit["_source"]["meta"]["source"]
        timestamp = hit["_source"]["meta"]["timestamp"]
        if re.search(r"wikipedia\.org", source):
            wikipedia_results.append({"hit": hit, "timestamp": timestamp})
    
    if wikipedia_results:
        latest_result = max(wikipedia_results, key=lambda x: x["timestamp"])
        return latest_result["hit"]
    else:
        return None


def process_latest_result(latest_result):
    filtered_data = {}
    if latest_result:
        for item in latest_result["_source"]["data"]:
            if item["data"] and item["name"]:
                filtered_data[item["name"]] = item["data"]        
        last_row_text = json.dumps(filtered_data, ensure_ascii=False, indent=2)
        return last_row_text[2:-2]
    return ""



# def extract_returning_sources(latest_result):
#     try:
#         returning_sources = eval(latest_result["_source"]["event"]["original"])["meta"]["source"]
#     except (KeyError, SyntaxError) as e:
#         returning_sources = "empty"
#     return returning_sources


def extract_returning_sources(latest_result):
    try:
        original_data = eval(latest_result["_source"]["event"]["original"])["meta"]["source"]

        target_name = 'image[0].source'
        returning_sources = [item['data'] for item in original_data if item['name'] == target_name]
        
        if not returning_sources:
            returning_sources = ["empty"]

    except (KeyError, SyntaxError) as e:
        returning_sources = ["empty"]
        original_data = []

    return returning_sources, original_data


In [133]:
response

wikidsata_results = []

for hit in response["hits"]["hits"]:
    source = hit["_source"]["meta"]["source"]
    timestamp = hit["_source"]["meta"]["timestamp"]
    if re.search(r"wikidata\.org", source):
        wikidsata_results.append({"hit": hit, "timestamp": timestamp})
    
    if wikidsata_results:
        latest_result = max(wikidsata_results, key=lambda x: x["timestamp"])

next((item['data']  for item in latest_result['hit']['_source']['data'] if item['name'] == 'image[0].source'), None)

In [21]:
import re

def filter_wikidata_results(response):
    wikidata_results = []

    for hit in response["hits"]["hits"]:
        source = hit["_source"]["meta"]["source"]
        timestamp = hit["_source"]["meta"]["timestamp"]

        if "wikidata.org" in source:
            wikidata_results.append({"hit": hit, "timestamp": timestamp})

    if wikidata_results:
        latest_result = max(wikidata_results, key=lambda x: x["timestamp"])
        picture_source = next(
            (item['data'] for item in latest_result['hit']['_source']['data'] if item.get('name') == 'image[0].source'),
            None
        )
        return picture_source
    
    return None


In [22]:
# next((item['data'] for item in response['hits']['hits'][1]['_source']['data'] if item['name'] == 'image[0].source'), None)
filter_wikidata_results(response)

'https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Laphonza_Butler_Senate_photo%2C_2023.jpg/220px-Laphonza_Butler_Senate_photo%2C_2023.jpg'

In [19]:
response

ObjectApiResponse({'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 4.0253515, 'hits': [{'_index': 'eye-raw-data', '_type': '_doc', '_id': '4e9nlo8BkSW7a5nmB-__', '_score': 4.0253515, '_ignored': ['event.original.keyword'], '_source': {'tags': ['eye-raw-data'], 'event': {'original': '{"meta":{"timestamp":1716215285,"source":"https://www.wikidata.org/wiki/Q121337973","name":"laphonza-butler","id":"Q121337973"},"data":[{"name":"instance of[0]","data":"human"},{"name":"image[0]","data":"Laphonza Butler Senate photo, 2023.jpg\\n5,805 × 7,256; 24.08 MB"},{"name":"image[0].source","data":"https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Laphonza_Butler_Senate_photo%2C_2023.jpg/220px-Laphonza_Butler_Senate_photo%2C_2023.jpg"},{"name":"image[0].point in time","data":"2023"},{"name":"signature[0]","data":"Laphonza Butler signature.svg\\n1,008 × 229; 15 KB"},{"name":"signatu

In [60]:
next((item['data'] for item in response['hits']['hits'][1]['_source']['data'] if item['name'] == 'image[0].source'), None)

'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Sen._John_Barrasso_Official_Portrait_7.17.07.jpg/220px-Sen._John_Barrasso_Official_Portrait_7.17.07.jpg'

In [96]:
for i in response:
    print(i)

took
timed_out
_shards
hits


In [131]:
response['hits']['hits'][4]

{'_index': 'eye-raw-data',
 '_type': '_doc',
 '_id': 'JxH4tI8BkSW7a5nmALd-',
 '_score': 1.5917382,
 '_ignored': ['event.original.keyword'],
 '_source': {'tags': ['eye-raw-data'],
  'event': {'original': '{"meta":{"timestamp":1716728102,"source":"https://www.wikidata.org/wiki/Q58106","name":"nikolay-mladenov","id":"Q58106"},"data":[{"name":"instance of[0]","data":"human"},{"name":"image[0]","data":"EPP Political Assembly, Nickolay Mladenov (cropped).jpg\\n1,129 × 1,448; 471 KB"},{"name":"image[0].source","data":"https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/EPP_Political_Assembly%2C_Nickolay_Mladenov_%28cropped%29.jpg/220px-EPP_Political_Assembly%2C_Nickolay_Mladenov_%28cropped%29.jpg"},{"name":"sex or gender[0]","data":"male"},{"name":"country of citizenship[0]","data":"Bulgaria"},{"name":"name in native language[0]","data":"Николай Евтимов Младенов(Bulgarian)"},{"name":"given name[0]","data":"Nikolai"},{"name":"family name[0]","data":"Mladenov"},{"name":"date of birth[0]",

In [125]:
response['hits']['hits'][1]['_source']['meta']

{'source': 'https://en.wikipedia.org/wiki/Barack_Obama',
 'id': 'Q76',
 'timestamp': 1716216088,
 'name': 'barack-obama'}

In [151]:
for i in jhon_bassaro['hits']['hits']:
    print(i)

{'_index': 'eye-raw-data', '_type': '_doc', '_id': 'IN-6h48BkSW7a5nmq9PV', '_score': 1.0498221, '_ignored': ['data.data.keyword', 'event.original.keyword'], '_source': {'tags': ['eye-raw-data'], '@timestamp': '2024-05-17T18:05:04.542345212Z', '@version': '1', 'meta': {'id': 'Q720521', 'timestamp': 1715958668, 'source': 'https://en.wikipedia.org/wiki/John_Barrasso', 'name': 'john-barrasso'}, 'event': {'original': '{"meta":{"timestamp":1715958668,"source":"https://en.wikipedia.org/wiki/John_Barrasso","name":"john-barrasso","id":"Q720521"},"data":[{"name":"","data":""},{"name":"Leader","data":"Mitch McConnell"},{"name":"Vice Chair","data":"Joni Ernst\\nShelley Moore Capito"},{"name":"Preceded by","data":"John Thune"},{"name":"Preceded by","data":"Joe Manchin"},{"name":"Preceded by","data":"Jim Inhofe"},{"name":"Succeeded by","data":"Tom Carper"},{"name":"Succeeded by","data":"Roy Blunt"},{"name":"Preceded by","data":"Jon Tester"},{"name":"Succeeded by","data":"John Hoeven"},{"name":"Prece

In [153]:
# 1716190643 1716190647
1716190647-1716190643

4

In [150]:
jhon_bassaro

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0498221, 'hits': [{'_index': 'eye-raw-data', '_type': '_doc', '_id': 'IN-6h48BkSW7a5nmq9PV', '_score': 1.0498221, '_ignored': ['data.data.keyword', 'event.original.keyword'], '_source': {'tags': ['eye-raw-data'], '@timestamp': '2024-05-17T18:05:04.542345212Z', '@version': '1', 'meta': {'id': 'Q720521', 'timestamp': 1715958668, 'source': 'https://en.wikipedia.org/wiki/John_Barrasso', 'name': 'john-barrasso'}, 'event': {'original': '{"meta":{"timestamp":1715958668,"source":"https://en.wikipedia.org/wiki/John_Barrasso","name":"john-barrasso","id":"Q720521"},"data":[{"name":"","data":""},{"name":"Leader","data":"Mitch McConnell"},{"name":"Vice Chair","data":"Joni Ernst\\nShelley Moore Capito"},{"name":"Preceded by","data":"John Thune"},{"name":"Preceded by","data":"Joe Manchin"},{"name":"Preceded by","data":"

In [104]:
filtered_data = {}
for i in response['hits']['hits'][1]['_source']['data']:
    if i['data'] and i['name']:
        filtered_data[i['name']] = i['data']

In [106]:
for key, value in filtered_data.items():
    print(f'{key}: {value}')

Vice President: Joe Biden
Preceded by: Alice Palmer
Succeeded by: Kwame Raoul
Born: Barack Hussein Obama II
 (1961-08-04) August 4, 1961 (age 62)
Honolulu, Hawaii, U.S.
Political party: Democratic
Spouse: Michelle Robinson ​(m. 1992)​
Children: MaliaSasha
Parents: Barack Obama Sr.Ann Dunham
Relatives: Obama family
Residence: Kalorama, Washington, D.C.
Education: At the age of six, Obama and his mother had moved to Indonesia to join his stepfather. From age six to ten, he was registered in school as "Barry"[34] and attended local Indonesian-language schools: Sekolah Dasar Katolik Santo Fransiskus Asisi (St. Francis of Assisi Catholic Elementary School) for two years and Sekolah Dasar Negeri Menteng 01 (State Elementary School Menteng 01) for one and a half years, supplemented by English-language Calvert School homeschooling by his mother.[35][36] As a result of his four years in Jakarta, he was able to speak Indonesian fluently as a child.[37] During his time in Indonesia, Obama's stepf

In [76]:
data_list = []
for i in response['hits']['hits'][2]['_source']['data']:
    data_list.append(i['data'])

# Або зберігаємо у рядок
data_string = "\n".join([i['data'] for i in response['hits']['hits'][2]['_source']['data']])


In [5]:
import re
from elasticsearch import Elasticsearch

es = Elasticsearch("http://10.167.65.50:9200")

def search_elasticsearch(index, query):
    response = es.search(index=index, body=query)
    return response

def filter_wikipedia_results(response):
    wikipedia_results = []
    for hit in response['hits']['hits']:
        source = hit['_source']['meta']['source']
        timestamp = hit['_source']['meta']['timestamp']
        if re.search(r'wikipedia\.org', source):
            wikipedia_results.append({'hit': hit, 'timestamp': timestamp})
    
    if wikipedia_results:
        latest_result = max(wikipedia_results, key=lambda x: x['timestamp'])
        return latest_result['hit']
    else:
        return None

def process_latest_result(latest_result):
    filtered_data = {}
    if latest_result:
        for item in latest_result['_source']['data']:
            if item['data'] and item['name']:
                filtered_data[item['name']] = item['data']
                
    last_row_text = json.dumps(filtered_data, ensure_ascii=False, indent=2)
    return last_row_text[2:-2]

def clean_data(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    cleaned_text = re.sub(r'\\u[0-9A-Fa-f]{4}', '', cleaned_text)
    cleaned_text = re.sub(r'\\xa0', ' ', cleaned_text)
    cleaned_text = re.sub(r'\u200b', '', cleaned_text)
    cleaned_text = cleaned_text.strip()
    return cleaned_text

In [4]:
import json
es_query = {
        "query": {
            "match": {
                "meta.id": "Q720521"
            }
        }
    }

new_res = search_elasticsearch(index="eye-raw-data", query=es_query)

NameError: name 'search_elasticsearch' is not defined

In [3]:
context = clean_data(process_latest_result(filter_wikipedia_results(new_res)))

In [4]:
eval(filter_wikipedia_results(new_res)['_source']['event']['original'])['meta']['source']

'https://en.wikipedia.org/wiki/John_Barrasso'

In [1]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WikipediaLoader
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from models import DataItem
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

llm = ChatOpenAI(model="gpt-4-0125-preview")

In [2]:
from langchain import hub
prompt_template = hub.pull("kaliani/generate_politicans")

def generate_politician_dossier(name, template, context):
    ## context = get_wiki_info(name) plan B

    parser = PydanticOutputParser(pydantic_object=DataItem)

    prompt = ChatPromptTemplate(
        messages=[
            HumanMessagePromptTemplate.from_template(
                template
            )
        ],
        input_variables=["context", "name"],
        partial_variables={
            "format_instructions": parser.get_format_instructions(),
        },
    )

    final_rag_chain = (
        prompt
        | llm
    )
    output = final_rag_chain.invoke({"context": context, "name": name})
    return output

In [3]:
for_res = generate_politician_dossier("John Anthony Barrasso", prompt_template.template, "Hello world!")

In [14]:
for_res

AIMessage(content='```json\n{\n  "Name": "John Anthony Barrasso",\n  "Position": "Сенатор США від штату Вайомінг",\n  "Country": "Сполучені Штати Америки",\n  "BirthDate": "21 липня 1952",\n  "Childhood": "-",\n  "Education": "Здобув ступінь бакалавра та медичний ступінь.",\n  "Career": "Працював ортопедичним хірургом перед виборами у Сенат.",\n  "PoliticalActivity": "Член Республіканської партії, сенатор з 2007 року.",\n  "Family": "-",\n  "ReturningSources": "empty"\n}\n```', response_metadata={'token_usage': {'completion_tokens': 204, 'prompt_tokens': 753, 'total_tokens': 957}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b891c365-4e3c-4b6f-ad64-603280fb3aa2-0')

In [4]:
main_info = json.loads(for_res.content[8:-3])

In [18]:
for_res.content

'```json\n{\n  "Name": "John Anthony Barrasso",\n  "Position": "Сенатор США від штату Вайомінг",\n  "Country": "Сполучені Штати Америки",\n  "BirthDate": "21 липня 1952",\n  "Childhood": "-",\n  "Education": "Здобув ступінь бакалавра та медичний ступінь.",\n  "Career": "Працював ортопедичним хірургом перед виборами у Сенат.",\n  "PoliticalActivity": "Член Республіканської партії, сенатор з 2007 року.",\n  "Family": "-",\n  "ReturningSources": "empty"\n}\n```'

In [10]:
main_info['ReturningSources'] = 'https://en.wikipedia.org/wiki/John_Barrasso'

In [5]:
main_info

{'Name': 'John Anthony Barrasso',
 'Position': 'Сенатор США від штату Вайомінг',
 'Country': 'США',
 'BirthDate': '21 липня 1952',
 'Childhood': '-',
 'Education': 'Доктор медичних наук, закінчив Георгетаунський університет',
 'Career': 'Ортопедичний хірург, член Сенату штату Вайомінг',
 'PoliticalActivity': 'Член Республіканської партії, сенатор з 2007 року',
 'Family': '-',
 'ReturningSources': 'empty',
 'PictureSource': 'empty'}

In [25]:
import time
from datetime import datetime

# Поточний час у форматі timestamp
current_time = date.now().timestamp()

# Для прикладу, виводимо значення
print(current_time)


NameError: name 'date' is not defined

In [7]:
from 

countries.py  countries_rag.ipynb  politicans_rag.ipynb  __pycache__/


In [3]:
from service import create_politician_json

In [ ]:


dossier = create_politician_json(main_info.get('Name'), 'Q720521', prompt_template.template)

In [11]:
dossier.model_dump()

{'id': 1,
 'wikidataid': 'Q720521',
 'title': 'John Anthony Barrasso III',
 'type': 0,
 'data': {'Name': 'John Anthony Barrasso III',
  'Position': 'старший сенатор Сполучених Штатів від Вайомінга',
  'Country': 'Сполучені Штати Америки',
  'BirthDate': '1952-07-21',
  'Childhood': "Народився у Рідінг, Пенсильванія. Батько був бетонщиком, який мав освіту дев'ятого класу. Італійське походження з боку обох дідусів і бабусь.",
  'Education': 'Випускник Георгтаунського університету, де отримав ступінь бакалавра наук з біології у 1974 році та ступінь доктора медицини у 1978 році. Пройшов інтернатуру в медичній школі Єльського університету.',
  'Career': 'Ортопедичний хірург у приватній практиці в Каспер, Вайомінг, з 1983 по 2007 рік. Президент Медичного товариства Вайомінгу. Лікар родео для Професійної асоціації ковбойських родео.',
  'PoliticalActivity': 'Член Республіканської партії. Сенатор штату Вайомінг з 2003 по 2007 рік. Старший сенатор США від Вайомінга з 2007 року. Голова Конференц

In [12]:
response = es.index(index="eye-clean-data", id=1, document=dossier.model_dump())

In [46]:
import json
import json
from pydantic import ValidationError
from models import PoliticianRequest, PoliticanDocs

# js = json.loads(politican_docs.json(indent=2))

try:
    document = PoliticanDocs(**dossier.model_dump())
    print(document)
except ValidationError as e:
    print(f"Validation Error: {e}")

id=1 wikidataid='Q720521' title='John Anthony Barrasso III' type=0 data=DataItem(Name='John Anthony Barrasso III', Position='старший сенатор від штату Вайомінг', Country='Сполучені Штати Америки', BirthDate='1952-07-21', Childhood="Народився і виріс у Рідінг, Пенсильванія. Син Луїзи М. (ні ДеСіско) та Джона Ентоні Баррассо молодшого. Батько був майстром з бетону, мав освіту дев'ятого класу. Баррасо є потомком італійських емігрантів з Каріфе та Васто.", Education="Закінчив Центральну католицьку вищу школу, яка пізніше об'єдналася з вищою школою Святого Імені, щоб створити Беркс Католіцьку Вищу Школу. Випускник Джорджтаунського університету з бакалавром наук у галузі біології та доктором медичних наук. Пройшов резидентуру в медичній школі Єльського університету.", Career='Приватна практика ортопеда у Каспер, Вайомінг, з 1983 по 2007 рік. Служив президентом медичного товариства Вайомінгу.', PoliticalActivity='Член Республіканської партії. Служив у Сенаті штату Вайомінг з 2003 по 2007 рік.

In [32]:
# from fastapi import FastAPI, HTTPException
# from pydantic import BaseModel
# import json
# # from elasticsearch_client import search_elasticsearch

# class QueryModel(BaseModel):
#     id: str

# async def search(query: QueryModel):
#     es_query = {
#         "query": {
#             "match": {
#                 "meta.id": query.id
#             }
#         }
#     }
    
#     try:
#         response = search_elasticsearch(index="eye-raw-data", query=es_query)
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=str(e))
    
#     latest_result = filter_wikipedia_results(response)
#     if not latest_result:
#         raise HTTPException(status_code=404, detail="No Wikipedia results found")
    
#     filtered_data = process_latest_result(latest_result)
    
#     # # Очищення даних
#     # cleaned_data = {key: clean_data(value) for key, value in filtered_data.items()}
    
#     return filtered_data

In [6]:
es_query = {
        "query": {
            "match": {
                "meta.id": "Q5658088"
            }
        }
    }

new_res = search_elasticsearch(index="eye-raw-data", query=es_query)

In [7]:
for i in new_res['hits']['hits']:
    print(i)

{'_index': 'eye-raw-data', '_type': '_doc', '_id': 'LWwzBpABkSW7a5nmrgRX', '_score': 3.7323647, '_ignored': ['event.original.keyword'], '_source': {'tags': ['eye-raw-data'], 'event': {'original': '{"meta":{"timestamp":1718090968,"source":"https://www.wikidata.org/wiki/Q5658088","name":"adolfo-orive-bellinger","id":"Q5658088"},"data":[{"name":"instance of[0]","data":"human"},{"name":"sex or gender[0]","data":"male"},{"name":"country of citizenship[0]","data":"Mexico"},{"name":"name in native language[0]","data":"Adolfo Orive Bellinger(Spanish)"},{"name":"given name[0]","data":"Adolfo"},{"name":"date of birth[0]","data":"13 August 1940"},{"name":"place of birth[0]","data":"Tijuana"},{"name":"father[0]","data":"Adolfo Orive Alba"},{"name":"languages spoken, written or signed[0]","data":"Spanish"},{"name":"occupation[0]","data":"politician"},{"name":"position held[0]","data":"Member of the Chamber of Deputies of Mexico"},{"name":"position held[0].parliamentary term","data":"LXII Legislatur

In [38]:
type(clean_data(process_latest_result(filter_wikipedia_results(new_res))))

str

In [137]:
raw_text = process_latest_result(filter_wikipedia_results(jhon_bassaro)).copy()
cleaned_data = {key: clean_data(value) for key, value in raw_text.items()}

In [51]:
print(json.dumps(raw_text, ensure_ascii=False, indent=2)[2:-2])

NameError: name 'raw_text' is not defined

In [9]:
# 'timestamp': 1716190647

import json
from datetime import datetime
from models import MetadataItem, PoliticanDocs
from utils import generate_politician_dossier
from service import get_context_and_sources, get_record_id_and_creation_time
from elastic import validate_document, index_document


prompt = hub.pull("kaliani/generate_politicans")


def create_politician_json(name: str, wikidataid: str, template):
    context, returning_sources, picture_source = get_context_and_sources(wikidataid)

    output = generate_politician_dossier(name, context, template)
    main_info = json.loads(output.content[8:-3])
    main_info["ReturningSources"] = returning_sources
    main_info["PictureSource"] = picture_source

    record_id, created_at = get_record_id_and_creation_time(wikidataid)

    updated_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    metadata_item = MetadataItem(
        sub_question="",
        answer=""
    )

    politician_docs = PoliticanDocs(
        id=record_id or wikidataid[1:], 
        wikidataid=wikidataid,
        title=name,
        type=1,
        data=main_info,
        metadata=[metadata_item],
        createdAt=created_at,
        updatedAt=updated_at
    )

    return politician_docs

ModuleNotFoundError: No module named 'pydantic'

In [8]:
create_politician_json("Adolfo Orive Bellinger", "Q5658088", prompt.template)

NameError: name 'create_politician_json' is not defined

In [16]:
prompt

PromptTemplate(input_variables=['context', 'format_instructions', 'name'], metadata={'lc_hub_owner': 'kaliani', 'lc_hub_repo': 'generate_politicans', 'lc_hub_commit_hash': '4b3047761db5c8cf5615e7308b552da9f36189390cae9c38b9b187be667680df'}, template="Ти співробітник посольства, який створює досьє на політиків світу згідно зі стандартним шаблоном.\n Для цього в тебе є приклад шаблону, який ти можеш використати для формування документа.\n '''\n{format_instructions}\n '''\n А також контекст, який ти можеш використати для формування документа.\n{context}\n Сформуй досьє {name}, використовуючи контекст та структуру.\n Якщо в контексті немає відповіді потрібної інформації, не намагайся її вигадати помічай такі пункти '-'.\n Твоя задача максимально - чітко дотримуватись структури та контексту.\n Відповідь згенеруй українською.")